### Zigbang 원룸 매물 데이터 수집

In [1]:
import pandas as pd
import requests

#### Process
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

#### 1. 동이름으로 위도 경도 구하기

##### 1) 웹페이지 분석 -> URL

url decode 해야함.

In [4]:
addr = '망원동' 
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'

##### 2) Request

In [5]:
response = requests.get(url) 
response

<Response [200]>

In [6]:
response.text

'{"success":true,"code":"200","items":[{"id":4002,"type":"address","name":"망원동","hint":"","description":"서울특별시 마포구 망원동","lat":37.556785583496094,"lng":126.9013442993164,"zoom":5,"polygon":[],"_score":null,"_source":{"name_length":3,"local1":"서울시","local2":"마포구","local3":"망원동","web_level":15,"web_lat":37.556785583496094,"web_lng":126.9013442993164,"app_level":15,"app_lat":37.556785583496094,"app_lng":126.9013442993164,"법정동코드":"1144012300"},"zoom_level":{"google":15,"daum":4},"zoom_level_v2":{"app":5,"web":4}}],"next":null,"limit":0}'

##### 3) JSON으로 위도 경도

In [7]:
data =response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat, lng

(37.556785583496094, 126.9013442993164)

#### 2. 위도 경도로 geohash 알아내기

- install geohash2 package

In [8]:
!pip install geohash2

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15554 sha256=652dc6a7cc8d690a0fea418180ceed6f158b98fe29b4beb678c2e93cc274bc9c
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\f6\7c\c4\1b3c6fea0ebc53bf730dc86bbee7a713d501455dfb4c1f0623
Successfully built geohash2


In [9]:
import geohash2

In [10]:
geohash = geohash2.encode(lat, lng, precision=5) # precision (영역크기) : 커질수록 영역이 작아짐

#### 3. geohash로 매물 아이디 가져오기

##### 1) 웹페이지 분석 -> URL

##### 2) Request

In [11]:
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
response = requests.get(url)
response

<Response [200]>

In [12]:
response.text

'{"clusters":[],"items":[{"lat":37.52906722773659,"lng":126.89979535013029,"item_id":37623131},{"lat":37.52909970073633,"lng":126.89979459312606,"item_id":37685329},{"lat":37.52906835690067,"lng":126.89979220341945,"item_id":37699264},{"lat":37.52907043063867,"lng":126.89895677025405,"item_id":37828417},{"lat":37.529114633593,"lng":126.8969544383019,"item_id":37595863},{"lat":37.52958116768765,"lng":126.89696559700613,"item_id":37812771},{"lat":37.52971666566198,"lng":126.90669680068928,"item_id":37594373},{"lat":37.529771171651106,"lng":126.89883157642349,"item_id":37779193},{"lat":37.52978651330546,"lng":126.89729902051963,"item_id":37681821},{"lat":37.52979599877884,"lng":126.89796592584005,"item_id":37789981},{"lat":37.5292838473765,"lng":126.89734002478102,"item_id":37812120},{"lat":37.529880627745094,"lng":126.9075465149246,"item_id":37591840},{"lat":37.529875332357385,"lng":126.90690391597163,"item_id":37647361},{"lat":37.529376455935015,"lng":126.9069080014477,"item_id":3777633

##### 3) JSON으로 매물ID

In [16]:
items = response.json()['items']
print(items[:5])
ids = [item['item_id'] for item in items]
len(ids), ids[:5]

[{'lat': 37.52906722773659, 'lng': 126.89979535013029, 'item_id': 37623131}, {'lat': 37.52909970073633, 'lng': 126.89979459312606, 'item_id': 37685329}, {'lat': 37.52906835690067, 'lng': 126.89979220341945, 'item_id': 37699264}, {'lat': 37.52907043063867, 'lng': 126.89895677025405, 'item_id': 37828417}, {'lat': 37.529114633593, 'lng': 126.8969544383019, 'item_id': 37595863}]


(581, [37623131, 37685329, 37699264, 37828417, 37595863])

#### 4. 매물 아이디로 매물 정보 가져오기

In [17]:
# 요청 id가 1000개 초과하면 에러 발생
url = 'https://apis.zigbang.com/v2/items/list'
params = {'domain': 'zigbang', 'item_ids':ids}
response = requests.post(url, params)
response

<Response [200]>

In [18]:
response.text

'{"items":[{"item_id":37623131,"section_type":null,"images_thumbnail":"https://ic.zigbang.com/ic/items/37623131/1.jpg","sales_type":"월세","sales_title":"월세","deposit":1000,"rent":50,"size_m2":19.83,"공급면적":{"m2":19.83,"p":"6"},"전용면적":{"m2":19.83,"p":"6"},"계약면적":null,"room_type_title":null,"floor":"2","floor_string":"2","building_floor":"9","title":"당산동에서 이보다 좋은 매물 못찾습니다","is_first_movein":null,"room_type":"02","status":true,"tags":["추천"],"service_type":"원룸","random_location":{"lat":37.529074152070415,"lng":126.90041974950167},"manage_cost":"7","reg_date":"2023-08-11T12:23:15+09:00","is_new":false,"addressOrigin":{"local1":"서울시","local2":"영등포구","local3":"당산동4가","local4":"","fullText":"서울시 영등포구 당산동4가","localText":"영등포구 당산동4가"},"action":{"isRead":false,"readAt":null,"isInquired":false,"inquiredAt":null,"isRewarded":false,"rewardedAt":null,"isReported":false,"reportedAt":null,"isChecked":false,"checkedAt":null,"isZzim":false},"contract":"","address":"영등포구 당산동4가","is_zzim":false,"address1":"서

In [28]:
data = response.json()['items']
df = pd.DataFrame(data)
df.tail(2)

,item_id,section_type,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,status,tags,service_type,random_location,manage_cost,reg_date,is_new,addressOrigin,action,contract,address,is_zzim,address1,address2,address3,item_bm_type
579,37703323,None,https://ic.zigbang.com/ic/items/37703323/1.jpg,월세,월세,5000,160,63.97,"{'m2': 63.97, 'p': '19.4'}","{'m2': 42.52, 'p': '12.9'}",None,None,3,3,6,전입o 가좌역 도보5분 살기좋은 신축쓰리룸,None,05,True,[추천],빌라,"{'lat': 37.56928138330394, 'lng': 126.90986972...",8,2023-08-12T09:50:21+09:00,False,"{'local1': '서울시', 'local2': '마포구', 'local3': '...","{'isRead': False, 'readAt': None, 'isInquired'...",,마포구 중동,False,서울시 마포구 중동,None,None,ZIGBANG
580,37836529,None,https://ic.zigbang.com/ic/items/37836529/1.jpg,월세,월세,5000,160,61.09,"{'m2': 61.09, 'p': '18.5'}","{'m2': 40.61, 'p': '12.3'}",None,None,고,고,6,가좌역 역세권 신축 첫입주 투룸 반전세주차O,None,04,True,[],오피스텔,"{'lat': 37.569294099503416, 'lng': 126.9092488...",8,2023-08-24T11:44:39+09:00,True,"{'local1': '서울특별시', 'local2': '마포구', 'local3':...","{'isRead': False, 'readAt': None, 'isInquired'...",,마포구 중동,False,서울시 마포구 중동,None,None,ZIGBANG


In [29]:
pd.options.display.max_columns = 40
df.tail(2)

,item_id,section_type,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,status,tags,service_type,random_location,manage_cost,reg_date,is_new,addressOrigin,action,contract,address,is_zzim,address1,address2,address3,item_bm_type
579,37703323,None,https://ic.zigbang.com/ic/items/37703323/1.jpg,월세,월세,5000,160,63.97,"{'m2': 63.97, 'p': '19.4'}","{'m2': 42.52, 'p': '12.9'}",None,None,3,3,6,전입o 가좌역 도보5분 살기좋은 신축쓰리룸,None,05,True,[추천],빌라,"{'lat': 37.56928138330394, 'lng': 126.90986972...",8,2023-08-12T09:50:21+09:00,False,"{'local1': '서울시', 'local2': '마포구', 'local3': '...","{'isRead': False, 'readAt': None, 'isInquired'...",,마포구 중동,False,서울시 마포구 중동,None,None,ZIGBANG
580,37836529,None,https://ic.zigbang.com/ic/items/37836529/1.jpg,월세,월세,5000,160,61.09,"{'m2': 61.09, 'p': '18.5'}","{'m2': 40.61, 'p': '12.3'}",None,None,고,고,6,가좌역 역세권 신축 첫입주 투룸 반전세주차O,None,04,True,[],오피스텔,"{'lat': 37.569294099503416, 'lng': 126.9092488...",8,2023-08-24T11:44:39+09:00,True,"{'local1': '서울특별시', 'local2': '마포구', 'local3':...","{'isRead': False, 'readAt': None, 'isInquired'...",,마포구 중동,False,서울시 마포구 중동,None,None,ZIGBANG


In [30]:
df.columns

Index(['item_id', 'section_type', 'images_thumbnail', 'sales_type',
       'sales_title', 'deposit', 'rent', 'size_m2', '공급면적', '전용면적', '계약면적',
       'room_type_title', 'floor', 'floor_string', 'building_floor', 'title',
       'is_first_movein', 'room_type', 'status', 'tags', 'service_type',
       'random_location', 'manage_cost', 'reg_date', 'is_new', 'addressOrigin',
       'action', 'contract', 'address', 'is_zzim', 'address1', 'address2',
       'address3', 'item_bm_type'],
      dtype='object')

In [31]:
columns = ['item_id', 'sales_type', 'deposit', 'rent', 'size_m2', 
          'floor', 'floor_string', 'building_floor', 'title',
          'service_type', 'manage_cost', 'address1']
df = df[columns]
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,floor_string,building_floor,title,service_type,manage_cost,address1
579,37703323,월세,5000,160,63.97,3,3,6,전입o 가좌역 도보5분 살기좋은 신축쓰리룸,빌라,8,서울시 마포구 중동
580,37836529,월세,5000,160,61.09,고,고,6,가좌역 역세권 신축 첫입주 투룸 반전세주차O,오피스텔,8,서울시 마포구 중동


In [32]:
df['address1']

0      서울시 영등포구 당산동4가
1      서울시 영등포구 당산동4가
2      서울시 영등포구 당산동4가
3      서울시 영등포구 당산동4가
4      서울시 영등포구 당산동4가
            ...      
576       서울시 마포구 성산동
577       서울시 마포구 성산동
578       서울시 마포구 성산동
579        서울시 마포구 중동
580        서울시 마포구 중동
Name: address1, Length: 581, dtype: object

In [34]:
result = df[df['address1'].str.contains('망원동')].reset_index(drop=True) 
result.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,floor_string,building_floor,title,service_type,manage_cost,address1
57,37804362,월세,300,72,19.83,2,2,3,신축급즉시입주깔끔원룸 주차가능,원룸,7,서울시 마포구 망원동
58,37811722,월세,2000,110,48.00,1,1,3,리모델링후 첫입주 쓰리룸월세 풀옵션,빌라,1,서울시 마포구 망원동


In [64]:
def oneroom(addr): 
    
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸' 
    response = requests.get(url) 
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng'] 
    geohash = geohash2.encode(lat, lng, precision=5) 
    
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸' 
    response = requests.get(url)
    items = response.json()['items'] 
    ids = [item['item_id'] for item in items]
    
    url = 'https://apis.zigbang.com/v2/items/list' 
    params = {'domain': 'zigbang', 'item_ids': ids} 
    response = requests.post(url, params) 
    data = response.json()['items'] 
    df = pd.DataFrame(data) 
    result = df[df['address1'].str.contains('망원동')].reset_index(drop=True) 
    return result[['item_id', 'title', 'deposit', 'rent', 'address1', 'size_m2', 'manage_cost']]

In [63]:
result = oneroom('마포구 합정동')
result.tail(2)

<Response [200]>


,item_id,title,deposit,rent,address1,size_m2,manage_cost
57,37804362,신축급즉시입주깔끔원룸 주차가능,300,72,서울시 마포구 망원동,19.83,7
58,37811722,리모델링후 첫입주 쓰리룸월세 풀옵션,2000,110,서울시 마포구 망원동,48.00,1


Spectial Command

In [38]:
%whos

Variable   Type         Data/Info
---------------------------------
addr       str          망원동
columns    list         n=12
data       list         n=581
df         DataFrame          item_id sales_type <...>\n[581 rows x 12 columns]
geohash    str          wydjx
geohash2   module       <module 'geohash2' from '<...>\\geohash2\\__init__.py'>
ids        list         n=581
items      list         n=581
lat        float        37.556785583496094
lng        float        126.9013442993164
oneroom    function     <function oneroom at 0x000001774A853E20>
params     dict         n=2
pd         module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests   module       <module 'requests' from '<...>\\requests\\__init__.py'>
response   Response     <Response [200]>
result     DataFrame         item_id sales_type  <...>         1  서울시 마포구 망원동  
url        str          https://apis.zigbang.com/v2/items/list


In [ ]:
%reset

##### 모듈 파일 만들기 

: 변수, 함수, 클래스를 하나의 파일(.py)로 묶어서 코드 작성 실행

In [70]:
%%writefile zigbang.py
import pandas as pd 
import requests 
import geohash2

def oneroom(addr): 
    
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸' 
    response = requests.get(url) 
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng'] 
    geohash = geohash2.encode(lat, lng, precision=5) 
    
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸' 
    response = requests.get(url)
    items = response.json()['items'] 
    ids = [item['item_id'] for item in items]
    
    url = 'https://apis.zigbang.com/v2/items/list' 
    params = {'domain': 'zigbang', 'item_ids': ids} 
    response = requests.post(url, params) 
    data = response.json()['items'] 
    df = pd.DataFrame(data) 
    result = df[df['address1'].str.contains('망원동')].reset_index(drop=True) 
    return result[['item_id', 'title', 'deposit', 'rent', 'address1', 'size_m2', 'manage_cost']]

Writing zigbang.py


In [67]:
%ls zigbang.py

 C 드라이브의 볼륨: WINDOWS
 볼륨 일련 번호: 1CE2-C115

 C:\Users\user\AIVLE\4_web\day1 디렉터리

2023-08-24  오후 05:07             1,061 zigbang.py
               1개 파일               1,061 바이트
               0개 디렉터리  165,159,800,832 바이트 남음
